# Notes:
<strong>- In this file, we select two Excel files that have the most common fields and are ideal for merging, as they are more likely to contain similar information. Extensive cleaning is applied to all fields to remove inconsistencies, including text fields, dates, etc.

<strong>- ID fields such as '#', '# 1', and 'Number' are not useful for connecting records across different files, so they can be removed. A unique ID can be created for each record at the end.

<strong>- Before applying the algorithm to remove inconsistencies in a field, we need to check its type (e.g., date, signed numbers). The expression "re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()" does not work for 'date' fields or values like '3.2' and '-3.2'.

# Check the merging part of the procedure on small dataframes to make sure it works

In [52]:
d1 = pd.DataFrame({'c1': [5,6,7,4,9], 'c2': [1,2,7,5,5], 'c3': ['a','b','v','t','t']})

In [53]:
d1

,c1,c2,c3
0,5,1,a
1,6,2,b
2,7,7,v
3,4,5,t
4,9,5,t


In [54]:
d2 = pd.DataFrame({'c2': [1,2, 2,3,5,66, 66], 'c3': ['a','b','b','v','t','yy','yy'], 'c4': [1.2,99,1.6,4.3,6.4,87.8, 100]})

In [55]:
d2

,c2,c3,c4
0,1,a,1.2
1,2,b,99.0
2,2,b,1.6
3,3,v,4.3
4,5,t,6.4
5,66,yy,87.8
6,66,yy,100.0


In [21]:
d2[['c2', 'c3']]

,c2,c3
0,1,a
1,2,b
2,2,b
3,3,v
4,5,t
5,66,yy
6,66,yy


In [56]:
d1[['c2', 'c3']].merge(d2[['c2', 'c3']], how='outer', on=['c2', 'c3'])

,c2,c3
0,1,a
1,2,b
2,2,b
3,7,v
4,5,t
5,5,t
6,3,v
7,66,yy
8,66,yy


In [23]:
d1[['c2', 'c3']].merge(d2[['c2', 'c3']], how='left', on=['c2', 'c3'])

,c2,c3
0,1,a
1,2,b
2,2,b
3,7,v
4,5,t
5,5,t


In [24]:
a = d1[['c2', 'c3']].merge(d2[['c2', 'c3']], how='outer', on=['c2', 'c3'])
a[~a.duplicated()]

,c2,c3
0,1,a
1,2,b
3,7,v
4,5,t
6,3,v
7,66,yy


In [59]:
d1.merge(d2, how='outer', on=['c2', 'c3'])[['c2', 'c3']]

,c2,c3
0,1,a
1,2,b
2,2,b
3,7,v
4,5,t
5,5,t
6,3,v
7,66,yy
8,66,yy


In [27]:
d1[~d1[['c2', 'c3']].duplicated()].shape

(4, 3)

In [28]:
d1.shape

(5, 3)

In [29]:
d2[~d2[['c2', 'c3']].duplicated()].shape

(5, 3)

In [ ]:
d2.shape

In [ ]:
d4 = pd.DataFrame(columns=['c2', 'c3'])

In [ ]:
d4.head()

In [ ]:
d1[['c2', 'c3']]

In [ ]:
d4 = pd.concat([d4, d1[['c2', 'c3']]])

In [ ]:
d4

In [ ]:
d4 = pd.concat([d4, d2[['c2', 'c3']]])

In [ ]:
d4

In [ ]:
d4[~d4.duplicated()]

In [ ]:
d3 = pd.DataFrame(columns=['c1', 'c2', 'c3', 'c4'])

In [ ]:
d3.head()

In [ ]:
d3 = pd.concat([d3, d1])

In [ ]:
d3

In [ ]:
d3 = pd.concat([d3, d2])

In [ ]:
d3

In [ ]:
d3[~d3.duplicated()]

# Helper function to convert different date formats to a consistent format.

In [1]:
def date_convertor(d):
    if d in [None, np.nan, 'NA/NA']:
        return d
    _ = re.sub(r'[^0-9/: -]+', '', str(d)).strip()
    try:
        _ = datetime.strptime(_, "%m/%d/%y")
    except:
        try:
            _ = datetime.strptime(_, "%m/%d/%Y")
        except:
            try:
                _ = datetime.strptime(_, '%Y-%m-%d %H:%M:%S')
            except:
                try:
                    _ = datetime.strptime(_, '%m/%Y')
                except:
                    print(f"couldn't convert {d}")
                    return np.nan
    new = _.strftime("%m/%d/%Y")
    return new

# Firstly, fix the inconsistencies in all columns in both files and then merge them. Fixing needs to be done based on achieved dictionaries from the initial merge.

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from datetime import datetime

In [3]:
pre = {}

In [4]:
# set the max columns to none in order to show all columns of dataframe
pd.set_option('display.max_columns', None)

In [5]:
file_names = ['00a_All drugs sorted alphabatically by their developers_1980-2016_cleaned_2.xlsx',
             '01c_Survival-terminated-Projects&Compounds_cleaned_2.xlsx',
             '04_All Recap clinical trials 1980-2016_cleaned_2.xlsx',
             'All drug projects Data_cleaned_2.xlsx',
             'All drug-alliance data_cleaned_2.xlsx',
             'All drugs sorted alphabatically by their developers_1980-2016_cleaned_2.xlsx',
             'List of marketed drugs_sales data required_cleaned_2.xlsx',
             'STG_TMNT_MKT_cleaned -Combined_cleaned_2.xlsx', 
             'US_Patents_June_2021_Updated_cleaned_2.xlsx']

In [6]:
df_All = pd.read_excel('All drug-alliance data_cleaned_2.xlsx')
# df_required = pd.read_excel('List of marketed drugs_sales data required_cleaned_2.xlsx')
df_Combined = pd.read_excel('STG_TMNT_MKT_cleaned -Combined_cleaned_2.xlsx')

In [7]:
df_All = df_All[~df_All.duplicated()]
df_Combined = df_Combined[~df_Combined.duplicated()]

In [8]:
df_All.shape, df_Combined.shape

((8927, 30), (8910, 29))

In [9]:
df_All.columns

Index(['#', 'Alliance', 'P1', 'P2', 'P Type', 'Date', 'Type', 'stage',
       'Developer', 'Firm code', 'Trade Name', 'Generic Name',
       'Product Identifier(s)', 'Current Status', 'Total Development',
       'Active Development Partner(s)', 'Former Developer(s)',
       'Therapeutic Area', 'Technology', 'Mechanism of Action',
       'Molecular Target', 'Target Action', 'Compound Origin', 'IND Date',
       'Project Start Date', 'error', 'indicator', 'pdfname', 'File Name',
       'Reason for Termination'],
      dtype='object')

In [10]:
df_Combined.columns

Index(['# 1', '# 2', 'Alliance', 'P1', 'P2', 'P Type', 'Date', 'Type', 'stage',
       'Developer', 'Firm code', 'Disease Indication', 'Trade Name',
       'Generic Name', 'Product Identifier(s)', 'Current Status', 'IND Date',
       'Project Start Date', 'Active Development Partner(s)',
       'Former Developer(s)', 'Therapeutic Area', 'Technology',
       'Mechanism of Action', 'Molecular Target', 'Target Action',
       'Compound Origin', 'pdfname', 'Total Development',
       'Reason for Termination'],
      dtype='object')

In [11]:
df_All = df_All.drop(['#'], axis=1)

In [12]:
df_Combined = df_Combined.drop(['# 1'], axis=1)

In [13]:
df_Combined = df_Combined.drop(['pdfname'], axis=1)

In [14]:
df_All = df_All.drop(['pdfname'], axis=1)

In [15]:
df_All = df_All[~df_All.duplicated()]
df_Combined = df_Combined[~df_Combined.duplicated()]

In [16]:
df_All.shape, df_Combined.shape

((8497, 28), (8910, 27))

In [17]:
common_cols = [c for c in list(df_All.columns) if c in list(df_Combined.columns)]

In [18]:
len(common_cols)

25

In [19]:
df_All_only_cols = [c for c in list(df_All.columns) if c not in list(df_Combined.columns)]

In [20]:
df_All[df_All_only_cols].head()

,error,indicator,File Name
0,s,NaN,stages
1,*,NaN,stages
2,s,NaN,stages
3,*,NaN,stages
4,s,NaN,stages


In [21]:
df_Combined_only_cols = [c for c in list(df_Combined.columns) if c not in list(df_All.columns)]

In [22]:
df_Combined[df_Combined_only_cols].head()

,# 2,Disease Indication
0,TMNT 3589,colorectal cancer
1,TMNT 3590,colorectal cancer
2,TMNT 3591,colorectal cancer
3,STG 167,cancer
4,STG 168,solid tumors


In [23]:
temp_c_c= common_cols

In [24]:
df_All_temp = df_All[~df_All[temp_c_c].duplicated()][temp_c_c]

In [25]:
df_Combined_temp = df_Combined[~df_Combined[temp_c_c].duplicated()][temp_c_c]

In [26]:
df_All_temp.shape, df_Combined_temp.shape

((8496, 25), (8496, 25))

In [27]:
df_resutl_temp = df_All_temp.merge(df_Combined_temp, how='outer', on=temp_c_c)

In [28]:
df_resutl_temp.shape

(9526, 25)

In [95]:
# df_resutl_temp[~df_resutl_temp.duplicated()].shape

(11683, 26)

# How to remove inconsistencies in two files before merging them:

1. Create a dictionary for the inconsistencies in each **common** column. This dictionary should be built based on an initial merge, i.e., merging only the common columns.
    - Identify repeated values in each column.
    - Check the type of each field and exclude 'Date' and 'Signed numbers' fields.

2. Create a dictionary for the inconsistencies in each **file-specific** column in both files.
    - Identify repeated values in each column.
    - Check the type of each field and exclude 'Date' and 'Signed numbers' fields.

3. The dictionary should now include all fields except those that did not pass the checks above. Use this dictionary to fix inconsistencies in both files.

4. Apply functions to clean 'Date' columns and 'Signed numbers' columns separately.

5. Merge the two cleaned files.


In [29]:
df_resutl_temp.columns

Index(['Alliance', 'P1', 'P2', 'P Type', 'Date', 'Type', 'stage', 'Developer',
       'Firm code', 'Trade Name', 'Generic Name', 'Product Identifier(s)',
       'Current Status', 'Total Development', 'Active Development Partner(s)',
       'Former Developer(s)', 'Therapeutic Area', 'Technology',
       'Mechanism of Action', 'Molecular Target', 'Target Action',
       'Compound Origin', 'IND Date', 'Project Start Date',
       'Reason for Termination'],
      dtype='object')

In [30]:
pre = {}
# Create a dictionary for the inconsistencies in each common column. Dictionaries needs to be 
# built based on the initial merge i.e. the merge of only common columns.

# the below list includs the 'date', 'singed numbers', etc.
unfixable_cols = ['Date', 'Current Status', 'Total Development', 'IND Date', 'Project Start Date']
for col in list(df_resutl_temp.columns):
    if col in unfixable_cols:
        print(col)
        continue
    pre[col] = {}
    pre[col]['values'] = list(df_resutl_temp.sort_values(['Product Identifier(s)'])[col])
    pre[col]['uniques'] = [str(c) for c in list(set(pre[col]['values']))]
    pre[col]['cleans'] = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[col]['uniques']]
#     Counter([pre[col]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) for c in pre[col]['uniques']])
    # 'L,O,S' and 'Lo,S' in the "Type" field are not the same.
    pre[col]['repeated'] = sorted([c for c in pre[col]['uniques'] \
                                 if (pre[col]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1
                                    and re.sub(r'[^a-zA-Z0-9]+', '', c).lower() != 'los' )])
    pre[col]['dic'] = {k:[] for k in set([re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[col]['repeated']])}
    
    for item in pre[col]['uniques']: 
        if re.sub(r'[^a-zA-Z0-9]+', '', item).lower() in pre[col]['dic'].keys(): 
            pre[col]['dic'][re.sub(r'[^a-zA-Z0-9]+', '', item).lower()].append(item)
    
    for i in pre[col]['dic'].keys(): 
        pre[col]['dic'][i] = pre[col]['dic'][i][0]
    
    # apply the line below for each column in both files.
    df_resutl_temp[col] = df_resutl_temp[col].apply(
        lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)

Date
Current Status
Total Development
IND Date
Project Start Date


In [31]:
for col in ['Date', 'IND Date', 'Project Start Date']:
    index = df_resutl_temp.columns.get_loc(col)
    print(index, col)
    df_resutl_temp.iloc[:, index] = df_resutl_temp.iloc[:, index].apply(date_convertor)

4 Date
couldn't convert 1 Proje
couldn't convert 34425
couldn't convert 1 Proje
22 IND Date
23 Project Start Date


In [32]:
df_resutl_temp.shape

(9526, 25)

In [33]:
df_resutl_temp[~df_resutl_temp.duplicated()].shape

(8624, 25)

In [34]:
# Complete the previous dictionary for the inconsistencies in each file's specific columns.

# the below list includs the 'date', 'singed numbers', etc.
unfixable_cols = []
for col in list(df_Combined[df_Combined_only_cols].columns):
    if col in unfixable_cols:
        print(col)
        continue
    pre[col] = {}
    pre[col]['values'] = list(df_Combined.sort_values(['Product Identifier(s)'])[col])
    pre[col]['uniques'] = [str(c) for c in list(set(pre[col]['values']))]
    pre[col]['cleans'] = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[col]['uniques']]
#     Counter([pre[col]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) for c in pre[col]['uniques']])
    # 'L,O,S' and 'Lo,S' in the "Type" field are not the same.
    pre[col]['repeated'] = sorted([c for c in pre[col]['uniques'] \
                                 if (pre[col]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1
                                    and re.sub(r'[^a-zA-Z0-9]+', '', c).lower() != 'los' )])
    pre[col]['dic'] = {k:[] for k in set([re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[col]['repeated']])}
    
    for item in pre[col]['uniques']: 
        if re.sub(r'[^a-zA-Z0-9]+', '', item).lower() in pre[col]['dic'].keys(): 
            pre[col]['dic'][re.sub(r'[^a-zA-Z0-9]+', '', item).lower()].append(item)
    
    for i in pre[col]['dic'].keys(): 
        pre[col]['dic'][i] = pre[col]['dic'][i][0]
    
    # apply the line below for each column in both files.
    df_Combined[col] = df_Combined[col].apply(
        lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)

In [35]:
len(pre.keys())

22

In [36]:
pre.keys()

dict_keys(['Alliance', 'P1', 'P2', 'P Type', 'Type', 'stage', 'Developer', 'Firm code', 'Trade Name', 'Generic Name', 'Product Identifier(s)', 'Active Development Partner(s)', 'Former Developer(s)', 'Therapeutic Area', 'Technology', 'Mechanism of Action', 'Molecular Target', 'Target Action', 'Compound Origin', 'Reason for Termination', '# 2', 'Disease Indication'])

In [37]:
len(pre.keys())

22

In [38]:
pre.keys()

dict_keys(['Alliance', 'P1', 'P2', 'P Type', 'Type', 'stage', 'Developer', 'Firm code', 'Trade Name', 'Generic Name', 'Product Identifier(s)', 'Active Development Partner(s)', 'Former Developer(s)', 'Therapeutic Area', 'Technology', 'Mechanism of Action', 'Molecular Target', 'Target Action', 'Compound Origin', 'Reason for Termination', '# 2', 'Disease Indication'])

# Check each field to see if should be added to the dictionary or not.

In [438]:
# temp_c_c= common_cols
# temp_c_c= df_All_only_cols
temp_c_c= df_Combined_only_cols

In [ ]:
# 'Date' is date, 
# 'Current Status' is ['Phase I/II', 'Phase III']
# 'Total Development' is ['-0.4', '-2.9', '-3.7', '-4.9', '-6.9', '0.4', '2.9', '3.7', '4.9', '6.9']
# 'IND Date' is date,
# 'Project Start Date' is date,
index = 2
temp_c_c[index]

In [452]:
# values = list(df_resutl_temp.sort_values(['Product Identifier(s)'])[temp_c_c[index]])
#values = list(df_All.sort_values(['Product Identifier(s)'])[temp_c_c[index]])
values = list(df_Combined.sort_values(['Product Identifier(s)'])[temp_c_c[index]])

In [453]:
values[0]

'bronchospasm'

In [454]:
len(values)

8910

In [455]:
uniques = [str(c) for c in list(set(values))]

In [456]:
len(uniques)

946

In [457]:
cleans = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in uniques]

In [458]:
len(list(set(cleans)))

911

In [459]:
repeated = sorted([c for c in uniques if (cleans.count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1
                                         and re.sub(r'[^a-zA-Z0-9]+', '', c).lower() != 'los' )])

In [460]:
repeated

["Alzheimer's Disease",
 "Alzheimer's disease",
 'B-Cell Malignancies',
 'B-cell malignancies',
 'Clostridium difficile infection',
 'Duchenne Muscular Dystrophy',
 'Duchenne muscular dystrophy',
 "Friedreich's Ataxia",
 "Friedreich's ataxia",
 'Graft-versus-Host Disease',
 'Hepatitis B Infection',
 'Hepatitis C virus infection',
 "Huntington's Disease",
 "Huntington's disease",
 'Japanese encephalitis',
 "Parkinson's Disease",
 "Parkinson's disease",
 "Waldenstrom's Macroglobulinemia",
 "Waldenstrom's macroglobulinemia",
 'West Nile virus',
 'age related macular degeneration',
 'age-related macular degeneration',
 "alzheimer's disease",
 'beta thalassemia',
 'beta-thalassemia',
 'clostridium difficile infection',
 'community acquired bacterial pneumonia',
 'community acquired pneumonia',
 'community-acquired bacterial pneumonia',
 'community-acquired pneumonia',
 'complicated intra abdominal infections',
 'complicated intra-abdominal infections',
 'cutaneous T cell lymphoma',
 'cutane

In [450]:
sorted(uniques)

['MKT 1',
 'MKT 10',
 'MKT 100',
 'MKT 1000',
 'MKT 1001',
 'MKT 1002',
 'MKT 1003',
 'MKT 1004',
 'MKT 1005',
 'MKT 1006',
 'MKT 1007',
 'MKT 1008',
 'MKT 1009',
 'MKT 101',
 'MKT 1010',
 'MKT 1011',
 'MKT 1012',
 'MKT 1013',
 'MKT 1014',
 'MKT 1015',
 'MKT 1016',
 'MKT 1017',
 'MKT 1018',
 'MKT 1019',
 'MKT 102',
 'MKT 1020',
 'MKT 1021',
 'MKT 1022',
 'MKT 1023',
 'MKT 1024',
 'MKT 1025',
 'MKT 1026',
 'MKT 1027',
 'MKT 1028',
 'MKT 1029',
 'MKT 103',
 'MKT 1030',
 'MKT 1031',
 'MKT 1032',
 'MKT 1033',
 'MKT 1034',
 'MKT 1035',
 'MKT 1036',
 'MKT 1037',
 'MKT 1038',
 'MKT 1039',
 'MKT 104',
 'MKT 1040',
 'MKT 1041',
 'MKT 1042',
 'MKT 1043',
 'MKT 1044',
 'MKT 1045',
 'MKT 1046',
 'MKT 1047',
 'MKT 1048',
 'MKT 1049',
 'MKT 105',
 'MKT 1050',
 'MKT 1051',
 'MKT 1052',
 'MKT 1053',
 'MKT 1054',
 'MKT 1055',
 'MKT 1056',
 'MKT 1057',
 'MKT 1058',
 'MKT 1059',
 'MKT 106',
 'MKT 1060',
 'MKT 1061',
 'MKT 1062',
 'MKT 1063',
 'MKT 1064',
 'MKT 1065',
 'MKT 1066',
 'MKT 1067',
 'MKT 1068'

In [429]:
sorted(cleans)

['market', 'stages', 'termination']

In [437]:
df_All[df_All_only_cols[2]].unique()

array(['stages', 'termination', 'market'], dtype=object)

# Apply dictionary to the files

In [39]:
for col in list(pre.keys()):
    if col in list(df_All.columns):
        index = df_All.columns.get_loc(col)
        print('df_All: ', index, col)
        df_All.iloc[:, index] = df_All.iloc[:, index].apply(
            lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
#         df_All[col] = df_All[col].apply(
#             lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    if col in list(df_Combined.columns):
        index = df_Combined.columns.get_loc(col)
        print('df_Combined: ', index, col)
        df_Combined.iloc[:, index] = df_Combined.iloc[:, index].apply(
            lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
#         df_Combined[col] = df_Combined[col].apply(
#             lambda x: pre[col]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)


df_All:  0 Alliance
df_Combined:  1 Alliance
df_All:  1 P1
df_Combined:  2 P1
df_All:  2 P2
df_Combined:  3 P2
df_All:  3 P Type
df_Combined:  4 P Type
df_All:  5 Type
df_Combined:  6 Type
df_All:  6 stage
df_Combined:  7 stage
df_All:  7 Developer
df_Combined:  8 Developer
df_All:  8 Firm code
df_Combined:  9 Firm code
df_All:  9 Trade Name
df_Combined:  11 Trade Name
df_All:  10 Generic Name
df_Combined:  12 Generic Name
df_All:  11 Product Identifier(s)
df_Combined:  13 Product Identifier(s)
df_All:  14 Active Development Partner(s)
df_Combined:  17 Active Development Partner(s)
df_All:  15 Former Developer(s)
df_Combined:  18 Former Developer(s)
df_All:  16 Therapeutic Area
df_Combined:  19 Therapeutic Area
df_All:  17 Technology
df_Combined:  20 Technology
df_All:  18 Mechanism of Action
df_Combined:  21 Mechanism of Action
df_All:  19 Molecular Target
df_Combined:  22 Molecular Target
df_All:  20 Target Action
df_Combined:  23 Target Action
df_All:  21 Compound Origin
df_Combined

In [40]:
for col in ['Date', 'IND Date', 'Project Start Date']:
    if col in list(df_All.columns):
        index = df_All.columns.get_loc(col)
        print('df_All: ', index, col)
        df_All.iloc[:, index] = df_All.iloc[:, index].apply(date_convertor)

    if col in list(df_Combined.columns):
        index = df_Combined.columns.get_loc(col)
        print('df_Combined: ', index, col)
        df_Combined.iloc[:, index] = df_Combined.iloc[:, index].apply(date_convertor)


df_All:  4 Date
couldn't convert 1 Proje
couldn't convert 34425
df_Combined:  5 Date
couldn't convert 1 Proje
df_All:  22 IND Date
df_Combined:  15 IND Date
df_All:  23 Project Start Date
df_Combined:  16 Project Start Date


In [41]:
df_resutl1 = df_All.merge(df_Combined, how='outer', on=common_cols)

In [42]:
df_resutl1.shape

(9057, 30)

In [43]:
df_resutl1[common_cols][~df_resutl1[common_cols].duplicated()].shape

(8624, 25)

In [44]:
df = df_resutl1[~df_resutl1.duplicated()]

In [45]:
df.shape

(9046, 30)

In [46]:
df.to_excel("All drug-alliance data_cleaned_2_and_STG_TMNT_MKT_cleaned -Combined_cleaned_2.xlsx", index=False)

In [15]:
common_cols = [c for c in list(df_alliance.columns) if c in list(df_sorted.columns)]

In [16]:
print(common_cols)

['Developer', 'Trade Name', 'Generic Name', 'Product Identifier(s)', 'Current Status', 'Active Development Partner(s)', 'Former Developer(s)', 'Therapeutic Area', 'Technology', 'Mechanism of Action', 'Compound Origin', 'IND Date', 'Project Start Date', 'Reason For Termination']


In [17]:
df_alliance_only_cols = [c for c in list(df_alliance.columns) if c not in list(df_sorted.columns)]

In [18]:
print(df_alliance_only_cols)

['Alliance', 'P1', 'P2', 'P.Type', 'Date', 'Type', 'stage', 'Firm.code', 'Total.Development', 'Molecular.Target', 'Target.Action', 'error', 'indicator', 'pdfname', 'File.Name']


In [19]:
df_sorted_only_cols = [c for c in list(df_sorted.columns) if c not in list(df_alliance.columns)]

In [20]:
print(df_sorted_only_cols)

['Disease Indication', 'Product Description', 'Stage at In-License', 'Date of In-License', 'Indication Ordinal', 'Stage at Termination', 'Termination Date']


In [21]:
temp_c_c= common_cols

In [22]:
#df.drop(['B', 'C'], axis=1)

In [23]:
df_alliance.shape, df_sorted.shape

((8927, 29), (3677, 21))

In [24]:
df_merged_init = df_alliance.merge(df_sorted, how='outer', on=temp_c_c)

In [25]:
df_merged_init.shape

(12604, 36)

### TEMP HEADER duplicate after merging even by considering only common fields

In [24]:
df_alliance_temp = df_alliance[~df_alliance[temp_c_c].duplicated()][temp_c_c]

In [25]:
df_sorted_temp = df_sorted[~df_sorted[temp_c_c].duplicated()][temp_c_c]

In [26]:
df_alliance_temp.shape, df_sorted_temp.shape

((3688, 14), (3631, 14))

In [27]:
df_resutl_temp = df_alliance_temp.merge(df_sorted_temp, how='outer', on=temp_c_c)

## Create a dictionary for the inconsistencies in each common column. Dictionaries needs to be built based on the initial merge.

In [28]:
for col in ['Developer', 'Trade Name', 'Generic Name', 'Product Identifier(s)', 'Active Development Partner(s)', 
            'Former Developer(s)', 'Therapeutic Area', 'Technology', 'Mechanism of Action', 'Compound Origin', 
            'Reason For Termination']:
    index = df_resutl_temp.columns.get_loc(col)
    print(index, col)
    pre[index] = {}
    pre[index]['values'] = list(df_resutl_temp.sort_values(['Product Identifier(s)'])[temp_c_c[index]])
    pre[index]['uniques'] = [str(c) for c in list(set(pre[index]['values']))]
    pre[index]['cleans'] = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[index]['uniques']]
#     Counter([pre[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) for c in pre[index]['uniques']])
    pre[index]['repeated'] = sorted([c for c in pre[index]['uniques'] \
                                 if pre[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1])
    pre[index]['dic'] = {k:[] for k in set([re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre[index]['repeated']])}
    
    for item in pre[index]['uniques']: 
        if re.sub(r'[^a-zA-Z0-9]+', '', item).lower() in pre[index]['dic'].keys(): 
            pre[index]['dic'][re.sub(r'[^a-zA-Z0-9]+', '', item).lower()].append(item)
    
    for i in pre[index]['dic'].keys(): 
        pre[index]['dic'][i] = pre[index]['dic'][i][0]
    
    df_resutl_temp[temp_c_c[index]] = df_resutl_temp[temp_c_c[index]].apply(
        lambda x: pre[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    

0 Developer
1 Trade Name
2 Generic Name
3 Product Identifier(s)
5 Active Development Partner(s)
6 Former Developer(s)
7 Therapeutic Area
8 Technology
9 Mechanism of Action
10 Compound Origin
13 Reason For Termination


### Don't run this section. 
Fix the inconsistencies in 'Current Status' in these two files.

In [ ]:
# {'phaseiii': ['Phase III', 'Phase I/II']}

In [ ]:
# index = 4

## Fixing needs to be done based on achieved dictionaries from the initial merge.

In [40]:
pre.keys()

dict_keys([0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 13])

In [31]:
for col in ['Developer', 'Trade Name', 'Generic Name', 'Product Identifier(s)', 'Active Development Partner(s)', 
            'Former Developer(s)', 'Therapeutic Area', 'Technology', 'Mechanism of Action', 'Compound Origin', 
            'Reason For Termination']:
    index = df_resutl_temp.columns.get_loc(col)
    print(index, col)
    df_alliance[col] = df_alliance[col].apply(
    lambda x: pre[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    
    df_sorted[col] = df_sorted[col].apply(
    lambda x: pre[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)


0 Developer
1 Trade Name
2 Generic Name
3 Product Identifier(s)
5 Active Development Partner(s)
6 Former Developer(s)
7 Therapeutic Area
8 Technology
9 Mechanism of Action
10 Compound Origin
13 Reason For Termination


In [41]:
# for index in pre.keys():
#     df_alliance_temp[temp_c_c[index]] = df_alliance_temp[temp_c_c[index]].apply(
#     lambda x: pre[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    
#     df_sorted_temp[temp_c_c[index]] = df_sorted_temp[temp_c_c[index]].apply(
#     lambda x: pre[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)


In [29]:
date_convertor('12/25/1994'), date_convertor('8/1/99')

('12/25/1994', '08/01/1999')

In [34]:
for col in ['IND Date', 'Project Start Date']:
    index = df_alliance.columns.get_loc(col)
    print(index, col)
    df_alliance.iloc[:, index] = df_alliance.iloc[:, index].apply(date_convertor)
    index = df_sorted.columns.get_loc(col)
    print(index, col)
    df_sorted.iloc[:, index] = df_sorted.iloc[:, index].apply(date_convertor)

22 IND Date
15 IND Date
23 Project Start Date
16 Project Start Date


## Create a dictionary for the inconsistencies in each df_alliance's column.

In [35]:
pre_df_alliance = {}

In [36]:
df_alliance_only = df_alliance[df_alliance_only_cols]

In [37]:
for col in ['Alliance', 'P1', 'P2', 'P.Type', 'Type', 'stage', 'Firm.code', 'Molecular.Target', 'Target.Action', 
            'error', 'indicator', 'File.Name']:
    index = df_alliance_only.columns.get_loc(col)
    print(index, col)
    pre_df_alliance[index] = {}
    pre_df_alliance[index]['values'] = list(df_alliance_only[df_alliance_only_cols[index]])
    pre_df_alliance[index]['uniques'] = [str(c) for c in list(set(pre_df_alliance[index]['values']))]
    pre_df_alliance[index]['cleans'] = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre_df_alliance[index]['uniques']]
#     Counter([pre_df_alliance[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) for c in pre_df_alliance[index]['uniques']])
    pre_df_alliance[index]['repeated'] = sorted([c for c in pre_df_alliance[index]['uniques'] \
                                 if pre_df_alliance[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1])
    pre_df_alliance[index]['dic'] = {k:[] for k in set([re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre_df_alliance[index]['repeated']])}
    
    for item in pre_df_alliance[index]['uniques']: 
        if re.sub(r'[^a-zA-Z0-9]+', '', item).lower() in pre_df_alliance[index]['dic'].keys(): 
            pre_df_alliance[index]['dic'][re.sub(r'[^a-zA-Z0-9]+', '', item).lower()].append(item)
            
    # 'los': ['L,O,S', 'Lo,S']
    if index == 5:
        pre_df_alliance[index]['dic'].pop('los')
    
    for i in pre_df_alliance[index]['dic'].keys(): 
        pre_df_alliance[index]['dic'][i] = pre_df_alliance[index]['dic'][i][0]
    df_alliance_only.iloc[:, index] = df_alliance_only.iloc[:, index].apply(
        lambda x: pre_df_alliance[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)

0 Alliance
1 P1
2 P2
3 P.Type
5 Type
6 stage
7 Firm.code
9 Molecular.Target
10 Target.Action
11 error
12 indicator
14 File.Name


### Don't run this section. 
Fix the inconsistencies in 'Total.Development' in df_alliance.

In [ ]:
# {'69': ['-6.9', '6.9'],  '37': ['3.7', '-3.7'],  'nan': ['nan',  'nan',  'nan', ...]}

In [255]:
# index = 8

### Don't run this section.
Fix the inconsistencies in 'pdfname' in df_alliance.

In [38]:
# index = 13

## Fixing for df_alliance specific columns.

In [39]:
for col in ['Alliance', 'P1', 'P2', 'P.Type', 'Type', 'stage', 'Firm.code', 'Molecular.Target', 'Target.Action', 
            'error', 'indicator', 'File.Name']:
    index = df_alliance_only.columns.get_loc(col)
    print(index, col)
    df_alliance[col] = df_alliance[col].apply(
    lambda x: pre_df_alliance[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    

0 Alliance
1 P1
2 P2
3 P.Type
5 Type
6 stage
7 Firm.code
9 Molecular.Target
10 Target.Action
11 error
12 indicator
14 File.Name


In [45]:
for col in ['Date',]:
    index = df_alliance.columns.get_loc(col)
    print(index)
    df_alliance.iloc[:, index] = df_alliance.iloc[:, index].apply(date_convertor)

4


## Create a dictionary for the inconsistencies in each df_sorted's column.

In [46]:
pre_df_sorted = {}

In [47]:
df_sorted_only = df_sorted[df_sorted_only_cols]

In [48]:
for col in ['Disease Indication', 'Product Description', 'Stage at In-License', 'Indication Ordinal', 
              'Stage at Termination']:
    index = df_sorted_only.columns.get_loc(col)
    print(index, df_sorted_only_cols[index])
    pre_df_sorted[index] = {}
    pre_df_sorted[index]['values'] = list(df_sorted_only[df_sorted_only_cols[index]])
    pre_df_sorted[index]['uniques'] = [str(c) for c in list(set(pre_df_sorted[index]['values']))]
    pre_df_sorted[index]['cleans'] = [re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre_df_sorted[index]['uniques']]
#     Counter([pre_df_sorted[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) for c in pre_df_sorted[index]['uniques']])
    pre_df_sorted[index]['repeated'] = sorted([c for c in pre_df_sorted[index]['uniques'] \
                                 if pre_df_sorted[index]['cleans'].count(re.sub(r'[^a-zA-Z0-9]+', '', c).lower()) > 1])
    pre_df_sorted[index]['dic'] = {k:[] for k in set([re.sub(r'[^a-zA-Z0-9]+', '', c).lower() for c in pre_df_sorted[index]['repeated']])}
    
    for item in pre_df_sorted[index]['uniques']: 
        if re.sub(r'[^a-zA-Z0-9]+', '', item).lower() in pre_df_sorted[index]['dic'].keys(): 
            pre_df_sorted[index]['dic'][re.sub(r'[^a-zA-Z0-9]+', '', item).lower()].append(item)
    
    for i in pre_df_sorted[index]['dic'].keys(): 
        pre_df_sorted[index]['dic'][i] = pre_df_sorted[index]['dic'][i][0]
    df_sorted_only.iloc[:, index] = df_sorted_only.iloc[:, index].apply(
        lambda x: pre_df_sorted[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)

0 Disease Indication
1 Product Description
2 Stage at In-License
4 Indication Ordinal
5 Stage at Termination


## Fixing for df_sorted specific columns.

In [49]:
for col in ['Disease Indication', 'Product Description', 'Stage at In-License', 'Indication Ordinal', 
              'Stage at Termination']:
    index = df_sorted_only.columns.get_loc(col)
    print(index, col)
    df_sorted[col] = df_sorted[col].apply(
    lambda x: pre_df_sorted[index]['dic'].get(re.sub(r'[^a-zA-Z0-9]+', '', str(x)).lower()) or x)
    

0 Disease Indication
1 Product Description
2 Stage at In-License
4 Indication Ordinal
5 Stage at Termination


In [50]:
for col in ['Date of In-License', 'Termination Date']:
    index = df_sorted.columns.get_loc(col)
    print(index, col)
    df_sorted.iloc[:, index] = df_sorted.iloc[:, index].apply(date_convertor)

14 Date of In-License
19 Termination Date


## Merge the cleaned files.

In [52]:
temp_c_c, len(temp_c_c)

(['Developer',
  'Trade Name',
  'Generic Name',
  'Product Identifier(s)',
  'Current Status',
  'Active Development Partner(s)',
  'Former Developer(s)',
  'Therapeutic Area',
  'Technology',
  'Mechanism of Action',
  'Compound Origin',
  'IND Date',
  'Project Start Date',
  'Reason For Termination'],
 14)

In [53]:
df_merged = df_alliance.merge(df_sorted, how='outer', on=temp_c_c)

In [54]:
df_merged.shape

(11451, 36)

In [ ]:
df_resutl_temp.sort_values(['Product.Identifier'])[30:40]#[['Alliance', 'Product.Identifier']]

In [ ]:
df_resutl_temp.sort_values(['Product.Identifier'])[20:30][['Alliance', 'Product.Identifier']]


In [ ]:
len(list(df_All_temp["Alliance"])), len(set(list(df_All_temp["Alliance"]))) 

In [ ]:
len(list(df_Combined_temp["Alliance"])), len(set(list(df_Combined_temp["Alliance"]))) 

In [ ]:
df_resutl_temp.shape

In [ ]:
df_resutl_temp[~df_resutl_temp[common_cols].duplicated()].shape

In [ ]:
common_cols.remove("pdfname")

In [ ]:
common_cols.remove("Alliance")

In [ ]:
df_resutl_temp = df_Combined_temp.merge(df_All_temp, how='inner', on=common_cols)

In [ ]:
df_resutl_temp.shape

In [ ]:
df_resutl_temp[~df_resutl_temp.duplicated()].shape

In [ ]:
common_cols.remove('#')

In [ ]:
common_cols

In [ ]:
df_All_temp2 = df_All[~df_All[common_cols].duplicated()][common_cols]

In [ ]:
df_All_temp2.shape

In [ ]:
df_All.shape, df_Combined.shape

In [ ]:
df_both = df_Combined.merge(df_All, how='outer', on=common_cols)

In [ ]:
df_both.shape

In [ ]:
df_both[~df_both.duplicated()].shape

In [ ]:
df_Combined.rename(columns = {'Reason.for.Termination':'Termination.reason'}, inplace = True)

In [ ]:
df_Combined.rename(columns = {'# 1':'#'}, inplace = True)

In [ ]:
df_Combined.drop(['B', 'C'], axis=1)


In [ ]:
common_cols = [c for c in list(df_Combined.columns) if c in list(df_All.columns)]

In [ ]:
len(common_cols)

In [ ]:
df_both = df_Combined.merge(df_All, how='outer', on=common_cols)

In [ ]:
df_both.shape

In [ ]:
df_both[~df_both.duplicated()].shape

In [ ]:
df_Combined[0:1]

In [ ]:
df_All[0:1]

In [ ]:
df_both[:1]

In [ ]:
df_both[-2:-1]

In [ ]:
print([c for c in list(df_All.columns) if c not in list(df_Combined.columns)])
print([c for c in list(df_Combined.columns) if c not in list(df_All.columns)])

In [ ]:
df_All.shape, df_Combined.shape

In [ ]:
df_All.nunique()

In [ ]:
df_All[~df_All[['Developer', 'Product.Identifier']].duplicated()].shape

In [ ]:
df_All['Developer'].duplicated()

In [ ]:
df_All.head(1)

In [ ]:
df_Combined.head(5)

In [ ]:
df_All['#'].values

In [ ]:
df_All.shape

In [ ]:
df_All[df_All['#'].isin(df_Combined['# 1'].values)].shape

In [ ]:
df_Combined.nunique()